# Libraries

In [18]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import text
import pandas as pd
import psycopg2
import os

In [19]:
load_dotenv()

True

# Database

In [20]:
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_DB = os.getenv('POSTGRES_DB')

In [21]:
engine = create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')

In [22]:
Session = sessionmaker(bind=engine)
session = Session()

# Data

In [24]:
sql_query = text("""
WITH global_climate_and_forest_fire AS (
    SELECT * FROM global_climate 
    JOIN forest_fire USING(latitude, longitude, \"date\")
), 
global_climate_and_forest_fire_and_ndvi AS (
    SELECT * FROM global_climate_and_forest_fire 
    JOIN ndvi USING(latitude, longitude, \"date\")
), land_cover_data AS (
    SELECT lc.latitude, lc.longitude, lc."year", 
    lcl."General class", lcl."class", lcl."Sub-class"
    FROM land_cover lc JOIN land_cover_legend lcl
    ON lc.land_cover = lcl."Map value"
),
population_density_and_land_cover AS (
    SELECT * FROM population_density 
    JOIN land_cover_data USING(latitude, longitude, \"year\")
), 
all_data AS (
    SELECT * FROM population_density_and_land_cover pd_lc
    JOIN global_climate_and_forest_fire_and_ndvi gc_ff_nd
    USING(latitude, longitude) 
    WHERE pd_lc.\"year\" = EXTRACT(YEAR FROM gc_ff_nd.\"date\")
)

SELECT * FROM all_data;
""")
data = session.execute(sql_query)

In [25]:
rows = data.fetchall()
df = pd.DataFrame(rows, columns=data.keys())

In [27]:
df.head(5)

,latitude,longitude,year,population_density,General class,class,Sub-class,date,ws,vpd,...,confidence,version,bright_t31,frp,daynight,type,n_pixels,vim,vim_avg,viq
0,10.0200,-73.3473,2002,13.845077,Terra Firma,Tree cover,5m trees,2002-08-02,2.67,1.46,...,75,6.03,301.3,23.0,D,0,60.0,0.6557,0.7418,89.1307
1,4.4798,-74.7071,2002,30.075151,Terra Firma,Dense short vegetation,100% short vegetation cover,2002-08-04,1.22,1.41,...,84,6.03,296.9,20.5,D,0,9.0,0.6566,0.6422,102.0808
2,1.1410,-77.8246,2002,16.361307,Terra Firma,Tree cover,25m trees,2002-08-07,3.01,0.28,...,50,6.03,286.5,29.7,D,0,17.0,0.7593,0.7629,99.5574
3,8.1353,-74.0259,2002,15.837952,Terra Firma,Tree cover,5m trees,2002-08-13,1.82,0.77,...,55,6.03,301.2,6.1,D,0,43.0,0.7754,0.7803,99.4157
4,8.6516,-73.4420,2002,10.572280,Terra Firma,Tree cover,8m trees,2002-08-13,2.17,0.62,...,88,6.03,304.1,32.9,D,0,59.0,0.8163,0.8328,98.1321


# Model

# Close connection

In [17]:
session.close()
engine.dispose()